In [1]:
import os
import pandas as pd
from pandas import read_csv
import numpy as np
import re
from io import StringIO

In [3]:
# ---- File Management ----
# CSV Models
metricasDeepGroup = read_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/metricasEstadOrderBySerie.csv",delimiter=",")
# Predictions made by model from the TS
predictionsListRavel_ML = np.load("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/predList_EST.npy", allow_pickle=True)
# Real Values for each TS
realList_ML = np.load("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/realList_EST.npy", allow_pickle=True)

metricasDeepGroup
#df = metricasDeepGroup.loc[metricasDeepGroup['id_serie'] == 39]
#df = df.loc[df['rep'] == 0]
#df = df.loc[df['output'] == 12]
#df

,Unnamed: 0,id_serie,rep,id_model,output,sMAPE,MAE,MAPE
0,16974,39,0,ets,3,0.031530,291.709435,0.061364
1,16975,39,0,theta,3,0.013476,125.549196,0.025677
2,16976,39,0,arima,3,0.028875,266.079690,0.055151
3,16977,39,1,ets,3,0.032587,295.163359,0.066302
4,16978,39,1,theta,3,0.035185,316.577140,0.070273
...,...,...,...,...,...,...,...,...
35995,15079,47993,1,theta,12,0.021406,231.228148,0.043433
35996,15080,47993,1,arima,12,0.010492,112.680278,0.021382
35997,15081,47993,2,ets,12,0.022660,214.390570,0.043016
35998,15082,47993,2,theta,12,0.016376,157.638819,0.032222


In [6]:
def metrics(pre, real):
    
    # sMAPE
    sMAPE=np.sum(np.abs(pre-real))/np.sum((pre+real))
    
    #MAE
    # Len en Real
    MAE=np.mean(np.abs(pre-real)/len(real))
        
    #MAPE
    MAPE=np.mean(np.abs((real-pre)/real))
    
    
    return [sMAPE, MAE, MAPE]
# Input: dataframe-> TS Dataframe, realValues-> Real Values for each TS, predictionValues -> Predictions made by model from the TS
# Output: List of real and predictions averages into a data frame

def averageDataListGenerator(dataframe, realValues, predictionValues):
    # List of averages
    temp_list = []
    temp_list_averages = []
    #Metrics List
    temp_metricsList = []
    #List of averages for NPY
    realNPY = []
    predNPY = []
    for i,row in dataframe.iterrows():
        print("  ")
        print("Iter #", i)
        print("  ")
        #print(row["id_serie"], row["id_model"], row["rep"], row["output"])
        df = dataframe.loc[dataframe['id_serie'] == row.loc["id_serie"]]
        df = df.loc[df['rep'] == row.loc["rep"]]
        df = df.loc[df['output'] == row.loc["output"]]
        realAve = []
        predicAve = []
        temp_list_averages = []
        
        for j,row in df.iterrows():
            print(row["id_serie"], row["rep"], row["output"], row["sMAPE"],row["MAE"],row["MAPE"])
            # Promediar arreglos 1 x 1.
            realAve.append(realValues[j])
            predicAve.append(predictionValues[j])
        
        
        realAve = calcAverages(realAve)
        predicAve = calcAverages(predicAve)
        temp_metricsList = metrics(realAve,predicAve)
        #print(metricsList)
        #Agregamos datos generales de la serie de tiempo
        temp_list_averages.append(row["id_serie"])
        temp_list_averages.append(row["rep"])
        temp_list_averages.append(row["output"])
        #Agregamos los promedios reales de la serie de tiempo
        #temp_list_averages.append(realAve)
        realNPY.append(realAve)
        #Agregamos los promedios de predicciones de la serie de tiempo
        #temp_list_averages.append(predicAve)
        predNPY.append(predNPY)
        #Agregamos las metricas calculadas con las pred y los reales de la serie de tiempo
        temp_list_averages.append(temp_metricsList[0])
        temp_list_averages.append(temp_metricsList[1])
        temp_list_averages.append(temp_metricsList[2])
        temp_list.append(temp_list_averages)
        #print("TS AVERAGES\n")
        #print(temp_list_averages)
        #print("TEMP LIST\n")
        #print(temp_list)
  
    # Create the pandas DataFrame
    df = pd.DataFrame(temp_list, columns=["id_serie","rep","output","sMAPE", "MAE", "MAPE"])
    df.to_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/EST_averages.csv", index = False, header=True)
    #np.save("real_assembly_ts.npy",np.array(realNPY))
    #np.save("pred_assembly_ts.npy",np.array(predNPY))
    return  

# Calculate Averages for a list
def calcAverages(averages):
    #print(averages)
    a = np.zeros(shape=(len(averages[0])), dtype='float32')
    
    for listAve in averages:
        for j in range(0,len(averages[0])):
            a[j] += listAve[j]
    
    return a/len(averages)
            
#averageDataListGenerator(metricasDeepGroup,realList_ML,predictionsListRavel_ML)
df = read_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/EST_averages.csv",delimiter=",")
df = df.drop_duplicates()
df

,id_serie,rep,output,sMAPE,MAE,MAPE
0,39,0,3,0.024595,75.926544,0.049372
3,39,1,3,0.029778,89.300080,0.059248
6,39,2,3,0.024624,76.619470,0.048547
9,39,0,12,0.040420,31.381273,0.078134
12,39,1,12,0.047002,36.775986,0.098512
...,...,...,...,...,...,...
35985,47993,1,3,0.004219,17.376032,0.008538
35988,47993,2,3,0.009566,40.611385,0.018834
35991,47993,0,12,0.016658,17.003782,0.033463
35994,47993,1,12,0.013185,11.771712,0.026841


In [12]:
# dataf: CSV DADO
# dataf2: CSV GEN
def averageDataListGenerator(dataf, dataf2):
    temp_df = []
    for i,row in dataf2.iterrows():
        values = []
        
        print("Iter",i)
        #print(row["id_serie"], row["rep"], row["output"],row["sMAPE"],row["MAE"],row["MAPE"])
        print("")
        df = dataf.loc[dataf['id_serie'] == row.loc["id_serie"]]
        df = df.loc[df['rep'] == row.loc["rep"]]
        df = df.loc[df['output'] == row.loc["output"]]
        df = df.groupby(['id_serie', 'output', 'rep']).mean()
        #print(df.head())
        
        #id_serie
        values.append(int(row["id_serie"]))
        #rep
        values.append(int(row["rep"]))
        #output
        values.append(int(row["output"]))
        #sMAPE
        sMAPE_df1 = float(df['sMAPE'])
        sMAPE_df2 = float(row.loc['sMAPE'])
        values.append(((sMAPE_df1 + sMAPE_df2)/2))
        #MAE
        MAE_df1 = float(df['MAE'])
        MAE_df2 = float(row.loc['MAE'])
        values.append(((MAE_df1 + MAE_df2)/2))
        #MAPE
        MAPE_df1 = float(df['MAPE'])
        MAPE_df2 = float(row.loc['MAPE'])
        values.append(((MAPE_df1 + MAPE_df2)/2))
        print(values)
        # Add to list
        temp_df.append(values)
           
    # Create the pandas DataFrame
    df = pd.DataFrame(temp_df, columns=["id_serie","rep","output","sMAPE", "MAE", "MAPE"])
    df.to_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/Final_averages_EST.csv", index = False, header=True)
    return


    
#averageDataListGenerator(metricasDeepGroup,df)
final = read_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/Estadisticos/Final_averages_EST.csv",delimiter=",")
final

,id_serie,rep,output,sMAPE,MAE,MAPE
0,39,0,3,0.024611,151.852992,0.048385
1,39,1,3,0.029765,178.600092,0.059417
2,39,2,3,0.024604,153.238958,0.049531
3,39,0,12,0.040812,207.742289,0.084533
4,39,1,12,0.047981,248.128828,0.092614
...,...,...,...,...,...,...
11995,47993,1,3,0.005341,48.637928,0.010750
11996,47993,2,3,0.010003,86.809655,0.019545
11997,47993,0,12,0.018129,128.476510,0.035545
11998,47993,1,12,0.014798,93.823539,0.029854
